# Parameterizing multi-component systems with Interchange

<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>
    The simplest way to install dependencies is to use the Interchange examples environment. From the root of the cloned openff-interchange repository:
    
    conda env create --name interchange-examples --file devtools/conda-envs/examples_env.yaml 
    conda activate interchange-examples
    pip install -e .
    cd examples
    jupyter notebook packed_box.ipynb
    
</details>

The OpenFF Toolkit does not currently provide facilities to prepare topologies from structure files containing multiple molecules. This limitation can be worked around by loading PDBs from OpenMM!

In [ ]:
import nglview
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.components._packmol import pack_box
from openff.interchange.drivers import get_summary_data

## Construct the Topology

The Toolkit provides a class called `Topology` which stores a system of molecules. It is similar to an `Interchange` in that it stores a list of molecules along with box vectors and some other system information, but unlike `Interchange` a topology is not associated with any force field or parameters. It provides a useful class method `from_openmm` which takes a multi-component OpenMM topology and produces the corresponding Toolkit topology.

Because OpenMM topologies include connectivity but not the rich chemical information stored in a `Molecule`, `from_openmm` needs a list of `Molecule` objects that describe the components of the topology. This method is therefore most useful for moving a system with coordinates into the OpenFF ecosystem, and not for identifying the components themselves.

First, we load a PDB file describing a mixed box of solvents into OpenMM. `propane_methane_butanol_0.2_0.3_0.5.pdb` is distributed with the Toolkit and describes a 3.5 nm cubic box filled with 20% propane, 30% methane, and 50% butanol. We can use OpenMM's PDB machinery to read the file:

The PDB file has locations and connectivity information, but to assign parameters we need information that doesn't exist in the format. OpenFF requires users to be explicit about what they're trying to do, rather than risk silently accepting malformed input. This allows us to error out as soon as something goes wrong, rather than let the user complete an expensive calculation on the wrong molecule.

As a result, when we pull system information into the OpenFF ecosystem, we must provide a list of molecules that are in the system. Box vectors, numbers of each molecule, and the ordering of atoms in the system are taken from the PDB file, but chemical identities are taken from the list. If two molecules in the list can't be distinguished in the PDB, or a molecule in the PDB can't be matched to any molecule in the list, the Toolkit raises an error that tells you there's a mistake. For a system of simple molecules like this, it's easy to specify the unique molecules with SMILES codes:

In [ ]:
molecules = [
    Molecule.from_smiles(smi)
    for smi in ["ClC(Cl)(Cl)Cl", "CC1=CC=CC=C1", "CS(=O)C", "CCCCCCO"]
]

In [ ]:
# TODO: Have this function return a Topology with box vectors, positions, and residue information set
trajectory, _ = pack_box(
    molecules=molecules,
    number_of_copies=4 * [20],
    mass_density=850 * unit.kilogram / unit.meter**3,
)

topology = Topology.from_mdtraj(trajectory.topology, unique_molecules=molecules)
topology.box_vectors = unit.Quantity(
    trajectory.unitcell_lengths[0],
    unit.nanometer,
)

topology.set_positions(unit.Quantity(trajectory.xyz[0], unit.nanometer))

In [ ]:
nglview.show_mdtraj(trajectory)

We can get the positions as an array from the PDB file object:

## Parameterize with Interchange

Now that we have a topology, we can build our `Interchange`!

In [ ]:
interchange = Interchange.from_smirnoff(
    force_field=ForceField("openff_unconstrained-2.0.0.offxml"), topology=topology
)

We can visualize it:

In [ ]:
interchange.visualize("nglview")

We can check that the box vectors from the PDB made it to the Interchange:

And we can calculate and compare energies with different MD engines!

In [ ]:
get_summary_data(interchange)